<a href="https://colab.research.google.com/github/nathencinoza/pyspark-movie-dataset/blob/main/S19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

S19) Top 10 3-uplas de palabras más populares en las descripciones de las películas removiendo stopwords.

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)
rdd = sqlContext.read.parquet('/content/drive/MyDrive/Orga 2/Movie dataset/Parquets/movies.parquet').rdd

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:114: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning,


In [ ]:
import nltk 
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

In [ ]:
rdd.first()

Row(adult='False', belongs_to_collection='Toy Story Collection', budget='30000000', genres='Animation,Comedy,Family', homepage='http://toystory.disney.com/toy-story', id='862', imdb_id='tt0114709', original_language='en', original_title='Toy Story', overview="Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.", popularity=21.946943, poster_path='/rhIRbceoE9lR4veEXuwCC2wARtG.jpg', production_companies='3', production_countries="[{'iso_3166_1': 'US', 'name': 'United States of America'}]", release_date='1995-10-30', revenue=373554033.0, runtime=81.0, spoken_languages="[{'iso_639_1': 'en', 'name': 'English'}]", status='Released', tagline=None, title='Toy Story', video=False, vote_average=7.7, vote_count=5415.0)

In [ ]:
rdd = rdd.filter(lambda x: x.overview != None)

In [ ]:
rdd = rdd.map(lambda x: (x.title, [w for w in (tokenizer.tokenize(x.overview.lower())) if w not in stopwords]))
rdd.take(1)

[('Toy Story',
  ['led',
   'woody',
   'andy',
   'toys',
   'live',
   'happily',
   'room',
   'andy',
   'birthday',
   'brings',
   'buzz',
   'lightyear',
   'onto',
   'scene',
   'afraid',
   'losing',
   'place',
   'andy',
   'heart',
   'woody',
   'plots',
   'buzz',
   'circumstances',
   'separate',
   'buzz',
   'woody',
   'owner',
   'duo',
   'eventually',
   'learns',
   'put',
   'aside',
   'differences'])]

In [ ]:
def ordenarTuplas(x):
  lista = list()
  for item in nltk.trigrams(x[1]): 
    lista.append((" ".join(item),1))
  return lista

Top 10 3-uplas:

In [ ]:
rdd.flatMap(lambda x: ordenarTuplas(x)).reduceByKey(lambda x,y: x+y).takeOrdered(10, lambda x: -x[1])

[('new york city', 410),
 ('world war ii', 397),
 ('based true story', 149),
 ('year old boy', 127),
 ('year old girl', 117),
 ('12 year old', 113),
 ('year old son', 108),
 ('year old daughter', 95),
 ('17 year old', 88),
 ('film tells story', 85)]